<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 랭체인 제공 도구를 기능으로 활용한 업무 해결

자동 생성은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 도구 또는 사람으로 구동되는 대화 가능한 에이전트를 제공합니다. 이 프레임워크는 다중 에이전트 대화를 통해 도구 사용과 사람 참여가 가능합니다. 이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)에서 확인할 수 있습니다.

이 노트북에서는 `AssistantAgent`와 `UserProxyAgent`를 사용하여 OpenAI 모델(모델 버전 0613)의 새로운 기능으로 함수를 호출하는 방법을 Langchain에서 제공하는 도구 및 툴킷 세트와 함께 시연하여 사용 가능한 35개 이상의 도구를 활용하는 방법을 보여드립니다.
에이전트를 초기화하려면 지정된 프롬프트와 함수 구성을 `AssistantAgent`에 전달해야 합니다. 해당 함수는 `AssistantAgent`가 호출하는 모든 함수를 실행할 `UserProxyAgent`에 전달되어야 합니다. 설명과 함수를 일치시켜야 하는 이 요구 사항 외에도, `AssistantAgent`의 시스템 메시지를 확인하여 지침이 함수 호출 설명과 일치하는지 확인하는 것이 좋습니다.

요구 사항 ## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 `MathUserProxyAgent`에서 함수를 가져올 것이므로 [mathchat] 옵션을 설치해 주세요:
```bash
pip install "pyautogen[mathchat]"
```
```bash
pip install Langchain
```

In [ ]:
%pip install "pyautogen[mathchat]~=0.1.0"

## API 엔드포인트 설정

`config_list_from_models`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_models) 함수는 제공된 모델 목록에 대해 Azure OpenAI 엔드포인트 및 OpenAI 엔드포인트를 사용하여 구성 목록을 생성하려고 시도합니다. 이 함수는 API 키와 API 베이스가 해당 환경 변수 또는 로컬 txt 파일에 저장되어 있다고 가정합니다:

- OpenAI API 키: os.environ["OPENAI_API_KEY"] 또는 `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API 키: os.environ["AZURE_OPENAI_API_KEY"] 또는 `aoai_api_key_file="key_aoai.txt"`. 한 줄에 하나씩 여러 개의 키를 저장할 수 있습니다.
- Azure OpenAI API 베이스: os.environ["AZURE_OPENAI_API_BASE"] 또는 `aoai_api_base_file="base_aoai.txt"`. 한 줄에 하나씩 여러 개의 베이스를 저장할 수 있습니다.

OpenAI API 키만 있거나 Azure OpenAI API 키 + 베이스만 있어도 괜찮습니다.
구글 콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 '파일 업로드' 아이콘을 선택해 파일을 업로드할 수 있습니다.

다음 코드는 일부 엔드포인트가 아직 기능을 지원하지 않기 때문에 구성 목록에서 Azure OpenAI 엔드포인트를 제외합니다. 엔드포인트가 있는 경우 `exclude` 인수를 제거하세요.

In [ ]:
import autogen

config_list = autogen.config_list_from_models(model_list=["gpt-4", "gpt-3.5-turbo", "gpt-3.5-turbo-16k"], exclude="aoai", openai_api_key_file="key_openai.txt")


## 함수 호출 만들기

이 예제에서는 `AssistantAgent`와 `UserProxyAgent`로 함수 호출을 실행하는 방법을 보여드리겠습니다. 기본 시스템 프롬프트인 `AssistantAgent`를 사용하여 LLM 어시스턴트가 코드로 작업을 수행할 수 있도록 하고, `UserProxyAgent`는 LLM 응답에서 코드 블록을 추출하여 이를 실행합니다. 새로운 "function_call" 기능을 사용하여 함수를 정의하고 `AssistantAgent`에 대한 OpenAI 설정에서 함수에 대한 설명을 지정합니다. 그런 다음 `UserProxyAgent`에 함수를 등록합니다.


In [ ]:
%pip install Langchain

In [ ]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type
import math

class CircumferenceToolInput(BaseModel):
    radius: float = Field()

class CircumferenceTool(BaseTool):
    name = "circumference_calculator"
    description = "Use this tool when you need to calculate a circumference using the radius of a circle"
    args_schema: Type[BaseModel] = CircumferenceToolInput

    def _run(self, radius: float):
        return float(radius) * 2.0 * math.pi

In [ ]:
from langchain.tools.file_management.read import ReadFileTool

# Define a function to generate llm_config from a LangChain tool
def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace (' ', '_'),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
      function_schema["parameters"]["properties"] = tool.args

    return function_schema

# Instantiate the ReadFileTool
read_file_tool = ReadFileTool()
custom_tool = CircumferenceTool()

# Construct the llm_config
llm_config = {
  #Generate functions config for the Tool
  "functions":[
      generate_llm_config(custom_tool),
      generate_llm_config(read_file_tool),
  ],
  "config_list": config_list,  # Assuming you have this defined elsewhere
  "request_timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)

# Register the tool and start the conversation
user_proxy.register_function(
    function_map={
        custom_tool.name: custom_tool._run,
        read_file_tool.name: read_file_tool._run,
    }
)

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy.initiate_chat(
    chatbot,
    message="Read the file with the path 'Test.txt', then calculate the circumference of a circle that has a radius of that files contents.", #" 7.81mm" in the file
    llm_config=llm_config,
)

user_proxy (to chatbot):

Read the file with the path 'Test.txt', then calculate the circumference of a circle that has a radius of that files contents.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: read_file *****
Arguments: 
{
  "file_path": "Test.txt"
}
**********************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION read_file...
user_proxy (to chatbot):

***** Response from calling function "read_file" *****
7.81mm
******************************************************

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: circumference_calculator *****
Arguments: 
{
  "radius": 7.81
}
*************************************************************

----------------------------------------------------------------------

# 파이스파크 예제

In [ ]:
%pip install pyspark

In [ ]:
#Starndard Langchain example
from langchain.agents import create_spark_sql_agent
from langchain.agents.agent_toolkits import SparkSQLToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities.spark_sql import SparkSQL
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
schema = "langchain_example"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema}")
spark.sql(f"USE {schema}")
csv_file_path = "./sample_data/california_housing_train.csv"
table = "california_housing_train"
spark.read.csv(csv_file_path, header=True, inferSchema=True).write.option("path", "file:/content/spark-warehouse/langchain_example.db/california_housing_train").mode("overwrite").saveAsTable(table)
spark.table(table).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

In [ ]:
# Note, you can also connect to Spark via Spark connect. For example:
# db = SparkSQL.from_uri("sc://localhost:15002", schema=schema)
spark_sql = SparkSQL(schema=schema)
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")
toolkit = SparkSQLToolkit(db=spark_sql, llm=llm)
agent_executor = create_spark_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

In [ ]:
#Starndard Langchain example
agent_executor.run("Describe the california_housing_train table")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: california_housing_train
Thought:I see that there is a table called "california_housing_train" in the database. I can use the "schema_sql_db" tool to get more information about this table.
Action: schema_sql_db
Action Input: "california_housing_train"
Observation: CREATE TABLE spark_catalog.langchain_example.california_housing_train (
  longitude DOUBLE,
  latitude DOUBLE,
  housing_median_age DOUBLE,
  total_rooms DOUBLE,
  total_bedrooms DOUBLE,
  population DOUBLE,
  households DOUBLE,
  median_income DOUBLE,
  median_house_value DOUBLE)
;

/*
3 rows from california_housing_train table:
longitude	latitude	housing_median_age	total_rooms	total_bedrooms	population	households	median_income	median_house_value
-114.31	34.19	15.0	5612.0	1283.0	1015.0	472.0	1.4936	66900.0
-114.47	34.4	19.0	7650.0	1901.0	1129.0	463.0	1.82	80100.0
-114.56	33.69	17.0	720.0	174.0	333.0	117.0	1.6509	85700.0
*/
Though

'The "california_housing_train" table has the following columns: longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, and median_house_value.'

In [ ]:
# LangChain direct tool usage instead of toolkit example
# from langchain.tools.spark_sql.tool import (
#     InfoSparkSQLTool,
#     ListSparkSQLTool,
#     QueryCheckerTool,
#     QuerySparkSQLTool,
# )
# debug_toolkit = [
#   QuerySparkSQLTool(db=spark_sql),
#   InfoSparkSQLTool(db=spark_sql),
#   ListSparkSQLTool(db=spark_sql),
#   QueryCheckerTool(db=spark_sql, llm=llm),
#]

In [ ]:

# Now use AutoGen with Langchain Tool Bridgre
tools = []
function_map = {}

for tool in toolkit.get_tools(): #debug_toolkit if you want to use tools directly
    tool_schema = generate_llm_config(tool)
    print(tool_schema)
    tools.append(tool_schema)
    function_map[tool.name] = tool._run

# Construct the llm_config
llm_config = {
  "functions": tools,
  "config_list": config_list,  # Assuming you have this defined elsewhere
  "request_timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)

print(function_map)

# Register the tool and start the conversation
user_proxy.register_function(
    function_map = function_map
)

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy.initiate_chat(
    chatbot,
    message="Describe the table names california_housing_train",
    llm_config=llm_config,
)

{'name': 'query_sql_db', 'description': '\n    Input to this tool is a detailed and correct SQL query, output is a result from the Spark SQL.\n    If the query is not correct, an error message will be returned.\n    If an error is returned, rewrite the query, check the query, and try again.\n    ', 'parameters': {'type': 'object', 'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': []}}
{'name': 'schema_sql_db', 'description': '\n    Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables.\n    Be sure that the tables actually exist by calling list_tables_sql_db first!\n\n    Example Input: "table1, table2, table3"\n    ', 'parameters': {'type': 'object', 'properties': {'table_names': {'title': 'Table Names', 'type': 'string'}}, 'required': []}}
{'name': 'list_tables_sql_db', 'description': 'Input is an empty string, output is a comma separated list of tables in the Spark SQL.', 'parameters': {'type': 'object'